<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Создание-выборок" data-toc-modified-id="Создание-выборок-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Создание выборок</a></span></li><li><span><a href="#Обучение-count_vectorizer" data-toc-modified-id="Обучение-count_vectorizer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение count_vectorizer</a></span></li><li><span><a href="#Функция-для-построения-и-оценивания-моделей-построенных-с-помощью-count-based-методов" data-toc-modified-id="Функция-для-построения-и-оценивания-моделей-построенных-с-помощью-count-based-методов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Функция для построения и оценивания моделей построенных с помощью count-based методов</a></span></li><li><span><a href="#Логистическая-регрессия-(count_vec)" data-toc-modified-id="Логистическая-регрессия-(count_vec)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Логистическая регрессия (count_vec)</a></span></li><li><span><a href="#Наивный-Байес-(count_vec)" data-toc-modified-id="Наивный-Байес-(count_vec)-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Наивный Байес (count_vec)</a></span></li><li><span><a href="#Обучение-tf-idf" data-toc-modified-id="Обучение-tf-idf-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Обучение tf-idf</a></span></li><li><span><a href="#Логистическая-регрессия-(tf-idf)" data-toc-modified-id="Логистическая-регрессия-(tf-idf)-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Логистическая регрессия (tf-idf)</a></span></li><li><span><a href="#Наивный-Байес-(tf-idf)" data-toc-modified-id="Наивный-Байес-(tf-idf)-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Наивный Байес (tf-idf)</a></span></li><li><span><a href="#DistilBert" data-toc-modified-id="DistilBert-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>DistilBert</a></span></li><li><span><a href="#Обучение-эмбеддингов-с-помощью-distilbert" data-toc-modified-id="Обучение-эмбеддингов-с-помощью-distilbert-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>Обучение эмбеддингов с помощью distilbert</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» c DistilBert

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Постройте модель со значением метрики качества *F1* не меньше 0.75** 


## Подготовка

In [5]:
import sys
import spacy
import pandas as pd
import string
import numpy as np
import torch
import transformers as ppb
import regex as re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from tqdm import notebook

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])

In [ ]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Сперва необходимо предобработать текст

- Удалить лишние пробелы 
- Приведем все слова к нижнему регистру
- Удалим знаки пунктуации и числа
- Удалим стоп слова
- Лемматизируем получившиеся предложения

In [7]:
punctuation = string.punctuation
stop_words = ENGLISH_STOP_WORDS
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def clear_text(col):
    string = col['text']
    string = " ".join(string.lower().strip().split())
    string = re.sub(r'\d+', '', string)
    string = " ".join(string.translate(str.maketrans('', '', punctuation)).split())
    string = ' '.join([word for word in string.split() if word not in stop_words])
    string = " ".join([token.lemma_ for token in lemmatizer(string)])
    return string

# Для берта удалим только знаки препинания и цифры и приведем к нижнему регистру
def clear_text_bert(col):
    string = col['text']
    string = " ".join(string.lower().strip().split())
    string = re.sub(r'\d+', '', string)
    string = " ".join(string.translate(str.maketrans('', '', punctuation)).split())
    return string


df['lemm_text'] = df.apply(clear_text, axis=1)

In [ ]:
# Удаляем апостроф из списка удаляемых элементов для берта
punctuation = re.sub(r"'", '', punctuation)

In [ ]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edit username hardcore metallica f...
1,D'aww! He matches this background colour I'm s...,0,daww match background colour I m seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really try edit war just guy const...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page that s
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,second time ask view completely contradict cov...
159447,You should be ashamed of yourself \n\nThat is ...,0,ashamed horrible thing talk page
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm there s actual article prostitutio...
159449,And it looks like it was actually you who put ...,0,look like actually speedy version delete look


## Обучение

Обучим несколько разных моделей и сравним их:
- Логистическая регрессия с count_vectorizer и tf-idf
- Наивный Байес с count_vectorizer и tf-idf
- Логистическая регрессия на обученных эмбедингах destilbert

### Создание выборок

In [ ]:
corpus = df['lemm_text']

x = corpus
y = df['toxic']

x_train, x_val_test, y_train, y_val_test = train_test_split(
      x, y, test_size = 0.4, random_state = 0)

x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size = 0.5, random_state = 0)

95575
31858
31859


### Обучение count_vectorizer

In [ ]:
cv = CountVectorizer(max_features=5000)

x_train_cv = cv.fit_transform(x_train)
x_val_cv = cv.transform(x_val)
x_test_cv = cv.transform(x_test)

### Функция для построения и оценивания моделей построенных с помощью count-based методов

In [ ]:
def count_based(x_train, y_train, x_val, y_val, x_test, y_test, clf):
    
    clf.fit(x_train, y_train)
    
    # Поиск лучшего порога на валидационной выборке
    predict = clf.predict_proba(x_val)[:, 1]
    best_thres = 0.01
    max_f_score = 0
    for i in range(1, 100):
        thres = i / 100
        var = f1_score(y_val, list(map(int, predict >= thres)))
        if var > max_f_score:
            max_f_score = var
            best_thres = thres
    
    # Оцениваем модель на тесте 
    predict = clf.predict_proba(x_test)[:, 1]
    max_f_score = f1_score(y_test, list(map(int, predict >= best_thres)))
    
    return max_f_score



### Логистическая регрессия (count_vec)

In [ ]:
lr_cv_f1 = count_based(x_train_cv, y_train, x_val_cv, y_val, x_test_cv, y_test, clf=LogisticRegression())

print('F1 на тестовой выборке модели логистической регрессии с помощью count_vec:', lr_cv_f1)

F1 на тестовой выборке модели логистической регрессии с помощью count_vec: 0.7620902304446607


### Наивный Байес (count_vec)

In [ ]:
nb_cv_f1 = count_based(x_train_cv, y_train, x_val_cv, y_val, x_test_cv, y_test, clf=MultinomialNB())
print('F1 на тестовой выборке модели наивного байеса с помощью count_vec:', nb_cv_f1)


F1 на тестовой выборке модели наивного байеса с помощью count_vec: 0.7289443441142668


### Обучение tf-idf

In [ ]:
tf = TfidfVectorizer(max_features=5000)

x_train_tf = tf.fit_transform(x_train)
x_val_tf = tf.transform(x_val)
x_test_tf = tf.transform(x_test)

### Логистическая регрессия (tf-idf)

In [ ]:
lr_tf_f1 = count_based(x_train_tf, y_train, x_val_tf, y_val, x_test_tf, y_test, clf=LogisticRegression())
print('F1 на тестовой выборке модели логистической регрессии с помощью tf-idf:', lr_tf_f1) 

F1 на тестовой выборке модели логистической регрессии с помощью tf-idf: 0.7743428665895189


### Наивный Байес (tf-idf)

In [ ]:
nb_tf_f1 = count_based(x_train_tf, y_train, x_val_tf, y_val, x_test_tf, y_test, clf=MultinomialNB())
print('F1 на тестовой выборке модели наивного байеса с помощью tf-idf:', nb_tf_f1)

F1 на тестовой выборке модели наивного байеса с помощью tf-idf: 0.7461325517016772


### DistilBert

In [8]:
df['lemm_text'] = df.apply(clear_text_bert, axis=1)

In [9]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
tokenized = df['lemm_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation=True)))

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
attention_mask = np.where(padded != 0, 1, 0)

### Обучение эмбеддингов с помощью distilbert

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 
torch.cuda.empty_cache() 


model.to(device)


embedings = torch.Tensor().to(device) 
batch_size = 30


for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            batch_embedings = model(batch, attention_mask=attention_mask_batch)
        
        embedings = torch.cat((embedings,batch_embedings[0][:,0,:]),0)

embedings = embedings.cpu().numpy()


  0%|          | 0/5310 [00:00<?, ?it/s]

In [14]:

x = embedings
y = df['toxic']

x_train, x_val_test, y_train, y_val_test = train_test_split(
      x, y, test_size = 0.4, random_state = 0)

x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size = 0.5, random_state = 0)

clf = LogisticRegression();
clf.fit(x_train, y_train)
 
predict = clf.predict_proba(x_val)[:, 1]
best_thres = 0.01
max_f_score = 0
for i in range(1, 100):
    thres = i / 100
    var = f1_score(y_val, list(map(int, predict >= thres)))
    if var > max_f_score:
        max_f_score = var
        best_thres = thres

# Оцениваем модель на тесте 
predict = clf.predict_proba(x_test)[:, 1]
max_f_score = f1_score(y_test, list(map(int, predict >= best_thres)))


print('F1 логистической регрессии на эмбедингах distilbert:', max_f_score)
 

F1 логистической регрессии на эмбедингах distilbert: 0.7384805054707966


## Выводы

В ходе проекта была проведена очистка текста:
- Удалены лишние пробелы 
- Приведены все слова к нижнему регистру
- Удалены знаки пунктуации и числа
- Удалены стоп слова
- Лемматизированы получившиеся слова в предложениях

Дальше были построены несколько различных моделей:
- Логистические регрессии с помощью count_vectorizer и tf-idf
- Модели наивного Байеса с помощью count_vectorizer и tf-idf
- Обучены эмбединги на модели distilbert (обучалась отдельно на gpu в google colab)

Лучший результат по метрике F1 показала модель логистической регрессии с помощью tf-idf (f1 = 0.77)
